# Introduction

In this notebook, we will implement the standard method of Collaborative Filtering with the Pareto Dominance. It is inspired by this paper: 
http://www.sciencedirect.com/science/article/pii/S0020025513002004

In [2]:
# Import the fucking useful libraries =)
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools

%matplotlib inline
%load_ext autoreload
%autoreload 2

# There's a lot of columns in the DF. 
# Therefore, we add this option so that we can see more columns
pd.options.display.max_columns = 100

In [7]:
# Define here the data set
dataset = '../data/data_train_10.csv'

# Load the data

In [13]:
data = pd.read_csv(dataset)
data['UserID'] = train['Id'].apply(lambda x: int(x.split('_')[0][1:]))
data['MovieID'] = train['Id'].apply(lambda x: int(x.split('_')[1][1:]))
data.head()


,Id,Prediction
0,r9_c4,5
1,r8_c4,5
2,r4_c4,5
3,r10_c4,3
4,r2_c4,4


In [14]:
data.shape

(96, 2)

# Define some Variables

In here, we will define some sets:
* $U = \left\{u\in\mathbb{N} \mid 1 \leq u \leq l \right\}$, the set of users
* $I = \left\{i \in\mathbb{N} \mid 1 \leq i \leq m \right\}$, the set of items